# RNN Model

This notebook contains the Recurrent Neural Network for generating playlists. It was built using layers provided by Keras.

To use this model, change the ```input_title``` variable in the last code box to the desired playlist title, then run all cells in the notebook. After the first run of the notebook, new playlists can be generated just by editing and running the last code box.

In [1]:
# importing necessary libraries
import numpy as np
import json
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [2]:
# open the file and load it as a json
f = open('spotify_million_playlist_dataset_challenge/challenge_set.json')
js = json.load(f)
playlists = js['playlists']

# process the json data and add to the appropriate list or dict
titles = []
tracks = []
trackID_to_name = {}

for playlist in playlists:
    if not playlist['tracks'] or 'name' not in playlist:
        continue
    titles.append(playlist['name'].lower())
    tracks.append(' '.join(track['track_uri'].split(":")[2] for track in playlist['tracks']))
    for track in playlist['tracks']:
        trackID_to_name[track['track_uri'].split(":")[2].lower()] = track['track_name']


In [3]:
# uses the keras tokenizer to tokenize a list of sentences
# returns the tokenized sentences and the tokenizer
def tokenize(sentences):
    text_tokenizer = Tokenizer()
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [4]:
# pad the tracks and titles
titles_tokens, title_tokenizer = tokenize(titles)
tracks_tokens, track_tokenizer = tokenize(tracks)

title_vocab = len(title_tokenizer.word_index) + 1
track_vocab = len(track_tokenizer.word_index) + 1

max_title_length = int(len(max(titles_tokens, key=len)))
max_track_length = int(len(max(tracks_tokens, key=len)))

pad_titles = pad_sequences(titles_tokens, max_title_length, padding = "post")
pad_tracks = pad_sequences(tracks_tokens, max_track_length, padding = "post")

pad_titles = pad_titles.reshape(*pad_titles.shape, 1)
pad_tracks = pad_tracks.reshape(*pad_tracks.shape, 1)

In [5]:
# create the layers of the modle
input_sequence = Input(shape=(max_title_length,))
embedding = Embedding(input_dim=title_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)
r_vec = RepeatVector(max_track_length)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(track_vocab))(decoder)

In [6]:
# creating the model with summary
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9)]               0         
                                                                 
 embedding (Embedding)       (None, 9, 128)            285440    
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVecto  (None, 100, 64)           0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 100, 64)           33024     
                                                                 
 time_distributed (TimeDist  (None, 100, 63997)        4159805   
 ributed)                                                    

In [7]:
# training
model_results = enc_dec_model.fit(pad_titles, pad_tracks, batch_size=30, epochs=3)

Epoch 1/3


2023-12-06 18:15:47.249474: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp_10.


234/234 [==============================] - 121s 506ms/step - loss: 5.9307 - accuracy: 0.6171
Epoch 2/3
234/234 [==============================] - 120s 512ms/step - loss: 4.4616 - accuracy: 0.6200
Epoch 3/3
234/234 [==============================] - 118s 506ms/step - loss: 4.4552 - accuracy: 0.6200


In [8]:
# takes the output sequence and converts it back to a sentence using the given tokenizer
def logits_to_sentence(logits, tokenizer):
    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = 'empty'

    return ' '.join([index_to_words[prediction] for prediction in np.argsort(logits, axis=1)[:, -1 * random.randrange(2, 10)]])

In [9]:
# uses the given title to generate a playlist for that title
def predict_tracklist(title, enc_dec_model, title_tokenizer, track_tokenizer, max_title_length, max_track_length):
    input_title_tokens = title_tokenizer.texts_to_sequences([title.lower()])
    pad_input_title = pad_sequences(input_title_tokens, max_title_length, padding="post")
    pad_input_title = pad_input_title.reshape(*pad_input_title.shape, 1)

    predictions = enc_dec_model.predict(pad_input_title)[0]

    predicted_sentence = logits_to_sentence(predictions, track_tokenizer)

    track_names = set()
    for track_id in predicted_sentence.split():
        track_names.add(trackID_to_name.get(track_id, "Unknown Track Name"))

    return predicted_sentence, track_names

# change input_title here to generate for other playlist titles
input_title = "summer"
predicted_sentence, track_names = predict_tracklist(input_title, enc_dec_model, title_tokenizer, track_tokenizer, max_title_length, max_track_length)

print(f"Input Title: {input_title}")
print(f"Predicted Tracklist: {predicted_sentence}")
print(f"Extracted Track Names: {track_names}")

1/1 [==============================] - 1s 554ms/step
Input Title: summer
Predicted Tracklist: 5dnfhmqgr128gmy2tc5cej 5dnfhmqgr128gmy2tc5cej 5dnfhmqgr128gmy2tc5cej 5dnfhmqgr128gmy2tc5cej 5dnfhmqgr128gmy2tc5cej 5dnfhmqgr128gmy2tc5cej 5dnfhmqgr128gmy2tc5cej 5dnfhmqgr128gmy2tc5cej 5dnfhmqgr128gmy2tc5cej 1xznggdreh1oqq0xzbwxa3 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 7kxjtscq5nl1loytl7xaws 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 5nqbuaeteogdd6hhcre0dz 05zackzw8ybrq3efgifsnb 05zackzw8ybrq3efgifsnb 05zackzw8ybrq3efgifsnb 05zackzw8